<h1> Finance Ethique </h1>

On a fichier un excel contenant les cours des différents actifs, et un autre contenant leur note ESG.

In [245]:
# - - import des différentes librairies utilisées - -

import pandas as pd
import numpy as np
from math import *
from scipy.optimize import minimize


On importe les données présentes dans les tableurs.

In [246]:
market_data = pd.read_csv("market_data.csv")
print('Données de marché:')
market_data.set_index("Date")

Données de marché:


,CSIQ,CHF,MCPHY.PA,TPIC,OTLY,CRBN.AS,ENPH,RDUS,EKOP.BR,ALCRB.PA,...,ZWS,CABKA.AS,BN.PA,AXS.L,TREX,ALHGR.PA,ALESE.PA,FREY,ACOMO.AS,HASI_y
Date,,,,,,,,,,,,,,,,,,,,,
1989-01-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.612835,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.612835,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-01-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.612835,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.312595,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11.258981,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-09,NaN,1.1429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-12,NaN,1.1417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-13,NaN,1.1270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
print('Notes ESG:')
vecteur_ESG = pd.read_csv("esg_data.csv")['Note'].values
vecteur_ESG

Notes ESG:


array([ 6. ,  7.5,  7.5,  4.5,  6. ,  7.5,  8. ,  7.5,  6. ,  8.6, 10. ,
        4. ,  5.5,  5.5,  8. ,  7.5,  6. ,  7. ,  5. , 10. ,  5.5,  6.5,
        3. ,  3. ,  4. ,  8. ,  7.5,  1.5,  2.5,  3. ,  6.5,  1.5,  5. ,
        4. ,  6. ,  2.5,  4. ,  8. ])

On calcule les rendements et volatilités des actifs, que l'on insère dans un nouveau tableau data contenant la note ESG, le rendement, et la volatilité de chaque actif.

In [248]:
result_data_benchmark = pd.DataFrame(columns=['Asset', 'Return', 'Volatility'])

returns = market_data.drop('Date', axis=1).pct_change(fill_method=None)
mean_returns = returns.mean()
mean_volatility = returns.std()

result_data_benchmark['Asset'] = mean_returns.index
result_data_benchmark['Return'] = mean_returns.values
result_data_benchmark['Volatility'] = mean_volatility.values

result_data_benchmark

,Asset,Return,Volatility
0,CSIQ,0.000905,0.046044
1,CHF,0.000101,0.006554
2,MCPHY.PA,0.000237,0.035495
3,TPIC,0.000696,0.049574
4,OTLY,-0.003046,0.052701
5,CRBN.AS,0.000162,0.017338
6,ENPH,0.002413,0.051683
7,RDUS,0.000700,0.030804
8,EKOP.BR,0.000298,0.017554
9,ALCRB.PA,0.000347,0.029340


On calcule la matrice de covariance pour pouvoir faire de la gestion indicielle.

In [249]:
cov_matrix = market_data.iloc[:, 1:40].pct_change(fill_method=None).dropna().cov()
cov_matrix


,CSIQ,CHF,MCPHY.PA,TPIC,OTLY,CRBN.AS,ENPH,RDUS,EKOP.BR,ALCRB.PA,...,ZWS,CABKA.AS,BN.PA,AXS.L,TREX,ALHGR.PA,ALESE.PA,FREY,ACOMO.AS,HASI_y
CSIQ,7.915447e-04,3.316042e-05,1.457903e-04,0.001184,0.000661,1.834501e-04,6.792817e-04,2.363757e-04,-4.189816e-05,2.065517e-04,...,1.324984e-04,7.416612e-07,4.057350e-05,-3.390569e-05,2.513230e-04,-8.717223e-05,-3.258603e-05,5.343311e-04,3.602214e-05,5.005044e-04
CHF,3.316042e-05,1.926277e-05,4.833344e-05,0.000190,0.000113,1.479114e-05,7.652407e-05,3.130813e-05,2.571717e-06,-6.883452e-06,...,2.372066e-05,3.069327e-05,8.534276e-07,1.320679e-05,4.070697e-05,3.831248e-08,1.528228e-06,4.051701e-05,-1.234546e-05,6.808875e-05
MCPHY.PA,1.457903e-04,4.833344e-05,1.139598e-03,0.000936,0.000310,1.804431e-04,2.007620e-04,1.429934e-04,-7.396994e-05,4.835984e-05,...,1.081851e-04,9.589847e-07,3.729530e-05,1.597234e-04,2.063705e-04,-3.721041e-06,7.480261e-07,2.639644e-04,2.249520e-05,3.827746e-04
TPIC,1.184291e-03,1.903116e-04,9.357473e-04,0.013351,0.002640,1.307986e-03,2.699802e-03,1.220252e-03,-7.866135e-05,5.316906e-04,...,6.058246e-04,-4.274938e-05,-1.617238e-04,-3.127785e-04,1.233179e-03,-8.811195e-05,-2.274916e-04,1.387063e-03,-4.316053e-05,1.721484e-03
OTLY,6.605879e-04,1.129790e-04,3.098312e-04,0.002640,0.005668,4.464983e-04,9.602632e-04,3.848239e-04,8.270494e-05,3.816053e-04,...,3.381679e-04,8.786315e-05,-7.130525e-05,-9.593972e-05,4.435271e-04,4.810435e-05,-1.082339e-04,1.169305e-03,-2.819620e-04,1.171696e-03
CRBN.AS,1.834501e-04,1.479114e-05,1.804431e-04,0.001308,0.000446,4.719637e-04,3.190891e-04,1.969346e-04,-3.498878e-05,1.995718e-04,...,7.210983e-05,3.607633e-05,-2.044119e-06,7.574279e-05,2.194491e-04,4.685023e-05,-4.038975e-05,2.691409e-04,-5.408980e-06,3.186273e-04
ENPH,6.792817e-04,7.652407e-05,2.007620e-04,0.002700,0.000960,3.190891e-04,1.675103e-03,4.263837e-04,-2.778729e-05,3.353346e-04,...,1.781316e-04,1.457918e-04,1.009809e-05,-1.192315e-04,4.267830e-04,-1.034185e-04,-8.403772e-05,6.635241e-04,-9.660876e-06,8.303466e-04
RDUS,2.363757e-04,3.130813e-05,1.429934e-04,0.001220,0.000385,1.969346e-04,4.263837e-04,5.595059e-04,-6.547279e-08,1.215184e-04,...,1.337998e-04,-6.559551e-06,-2.209974e-05,5.574838e-05,2.134799e-04,9.295695e-05,-1.303543e-04,3.284500e-04,-1.331296e-05,3.739334e-04
EKOP.BR,-4.189816e-05,2.571717e-06,-7.396994e-05,-0.000079,0.000083,-3.498878e-05,-2.778729e-05,-6.547279e-08,3.548795e-04,-5.988196e-05,...,-2.021847e-05,5.552078e-05,-2.968685e-05,2.166570e-05,-3.207330e-05,5.210780e-05,-6.852559e-05,1.323201e-04,-4.995380e-06,-1.238519e-04
ALCRB.PA,2.065517e-04,-6.883452e-06,4.835984e-05,0.000532,0.000382,1.995718e-04,3.353346e-04,1.215184e-04,-5.988196e-05,5.456772e-04,...,7.245910e-06,4.141982e-05,2.461046e-05,-4.403858e-05,8.825553e-05,3.837659e-06,-3.023100e-05,2.654753e-04,-2.274044e-05,2.679160e-04


In [250]:

# Sélectionner les colonnes se terminant par "Return"
return_columns = [col for col in market_data.columns if col.endswith("Return")]
# Calculer les rendements quotidiens pour toutes les colonnes sélectionnées
daily_returns = market_data[return_columns].pct_change()

# Renommer les colonnes pour les distinguer dans le nouveau DataFrame
daily_returns.columns = [f'Daily_{col}' for col in daily_returns.columns]

return_columns
# daily_returns

[]

Pour calculer la VaR empirique on utilise cette fonction :

In [251]:
def empirical_var(x, confidence_level=0.95):
    # Calculate the portfolio return as the weighted sum of individual asset returns
    portfolio_return = np.dot(market_data.iloc[:, 1:-1], x[:-1])
    portfolio_return = portfolio_return[~np.isnan(portfolio_return)]
    # Sort portfolio returns in ascending order
    # sorted_returns = np.sort(portfolio_return)

    # Calculate the index corresponding to the confidence level
    index = int((1 - confidence_level) * len(portfolio_return))

    # VaR corresponds to the negative of the return at the index
    var = -portfolio_return[index]

    return var

In [252]:
var_seuil = -10
esg_seuil = 6

X_init = np.zeros(len(cov_matrix.values))

# Définir la contrainte linéaire
contrainte_lineaire = {'type': 'eq', 'fun': lambda x: x[-1] +1}

# Définir la contrainte pour que chaque variable soit > 0
contrainte_positive = {'type': 'ineq', 'fun': lambda x: x[:-1]}

# Définir la contrainte pour que chaque variable soit > 0
contrainte_esg = {'type': 'ineq', 'fun': lambda x: np.dot(x[:-1],vecteur_ESG) - esg_seuil}

# Contrainte pour que la somme des variables soit égale à 1
contrainte_somme_un = {'type': 'eq', 'fun': lambda x: np.sum(x) - 0}

# Contrainte pour que la VaR soit plus petite que la valeur seuil
contrainte_seuil = {'type': 'ineq', 'fun': lambda x: var_seuil - empirical_var(x)}

# Combiner toutes les contraintes
contraintes = [contrainte_lineaire, contrainte_positive, contrainte_esg, contrainte_somme_un, contrainte_seuil]

# Définir la fonction objective
objective = lambda x: np.dot(x, np.dot(cov_matrix.values, x))

# Minimiser la fonction objective sous contrainte
result = minimize(objective, X_init, constraints=contraintes)

# Les valeurs optimales des variables se trouvent dans result.x
variables_optimales = result.x[:-1]

# La dernière valeur est la valeur fixée
valeur_fixee = result.x[-1]


In [253]:
print("Portefeuille minimisant la tracking error:")
result.x

Portefeuille minimisant la tracking error:


array([ 1.35771763e-18,  1.23499716e-18,  2.06032816e-18,  4.06565908e-19,
        7.41189698e-18,  3.34363863e-18,  6.16916075e-18,  2.95445092e-18,
       -1.15180940e-18,  2.28999799e-03,  4.14714090e-02, -1.12579034e-19,
       -7.29096520e-20, -3.19039039e-20,  3.88578774e-02, -3.00514838e-18,
       -2.16800270e-19, -5.12553084e-18, -9.24320446e-19,  4.46148932e-02,
        8.69418641e-01,  3.36202971e-19, -3.24612457e-19,  1.52575673e-18,
        4.82501277e-18,  5.86964848e-19,  1.72649384e-18,  1.05548084e-18,
        8.08934163e-19,  1.56026269e-19,  5.52943108e-18, -6.72830469e-19,
        1.51466769e-19, -2.73801787e-18, -1.15012908e-19,  9.14149047e-19,
       -7.90853106e-19,  3.34718119e-03, -1.00000000e+00])

In [254]:
print("Note ESG du portefeuille : ")
np.dot(variables_optimales,vecteur_ESG)

Note ESG du portefeuille : 


5.999999999990043

In [255]:
print("Tracking Error : ")
np.dot(result.x, np.dot(cov_matrix.values, result.x))

Tracking Error : 


2.2588997198709005e-05

In [256]:
print("Empirique VaR : ")
empirical_var(result.x)

Empirique VaR : 


-27.214476736353408

In [257]:
somme_depart = 1000

# Supposons que market_data est votre DataFrame et v est votre vecteur
index_values = market_data.columns[1:-1]  # Index de market_data sans la colonne de date

# Création du DataFrame avec l'index et votre vecteur
invest = pd.DataFrame([index_values, (result.x * somme_depart)[:-1]], index=['Index', 'Valeur'])
invest


,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
Index,CSIQ,CHF,MCPHY.PA,TPIC,OTLY,CRBN.AS,ENPH,RDUS,EKOP.BR,ALCRB.PA,...,HAIN,ZWS,CABKA.AS,BN.PA,AXS.L,TREX,ALHGR.PA,ALESE.PA,FREY,ACOMO.AS
Valeur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,2.289998,...,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,3.347181
